<a href="https://colab.research.google.com/github/MendesEric/ICTrab/blob/main/TESTENormalize_de_MERCapsuleNetPytorchADAMP9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Net Configuration

In [ ]:
import torch


def squash(inputs, axis=-1):
    """
    Do squashing
    :param inputs (tensor): tensor of dim [M, num_capsules, capsule_dim]
    :param axis: the dim to apply squash
    :return:
        (tensor): squashed tensor of dim [M, num_capsules, capsule_dim]
    """
    norm = torch.norm(inputs, p=2, dim=axis, keepdim=True)
    scale = norm ** 2 / (1 + norm ** 2) / (norm + 1e-8)
    return scale * inputs

In [ ]:
import os
import sys
import torch
from torch.optim import Adam, SGD, lr_scheduler
from torchvision import transforms
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, f1_score
from sklearn.utils import shuffle
import pickle
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

In [ ]:
class PrimaryCapsule(nn.Module):
    """
    Apply Conv2D with `out_channels` and then reshape to get capsules
    :param in_channels: input channels
    :param out_channels: output channels
    :param dim_caps: dimension of capsule
    :param kernel_size: kernel size
    :return: output tensor, size=[batch, num_caps, dim_caps]
    """
    def __init__(self, in_channels, out_channels, dim_caps, kernel_size, stride=1, padding=0):
        super(PrimaryCapsule, self).__init__()
        self.dim_caps = dim_caps
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)

    def forward(self, x):
        outputs = self.conv2d(x)
        outputs = outputs.permute(0, 2, 3, 1).contiguous() # I add
        outputs = outputs.view(x.size(0), -1, self.dim_caps)
        return squash(outputs)


In [ ]:
class MECapsule(nn.Module):
    """
    The ME capsule layer. Dense layer has `in_num` inputs, each is a scalar, the
    output of the neuron from the former layer, and it has `out_num` output neurons.
    MECapsule just expands the output of the neuron from scalar to vector.
    :param in_num_caps: number of cpasules inputted to this layer
    :param in_dim_caps: dimension of input capsules
    :param out_num_caps: number of capsules outputted from this layer
    :param out_dim_caps: dimension of output capsules
    :param routings: number of iterations for the routing algorithm
    """
    def __init__(self, in_num_caps, in_dim_caps, out_num_caps, out_dim_caps, routings=3):
        super(MECapsule, self).__init__()
        self.in_num_caps = in_num_caps
        self.in_dim_caps = in_dim_caps
        self.out_num_caps = out_num_caps
        self.out_dim_caps = out_dim_caps
        self.routings = routings
        self.weight = nn.Parameter(0.01 * torch.randn(out_num_caps, in_num_caps, out_dim_caps, in_dim_caps))

    def forward(self, x):
        x_hat = torch.squeeze(torch.matmul(self.weight, x[:, None, :, :, None]), dim=-1)
        x_hat_detached = x_hat.detach()

        b = torch.zeros(x.size(0), self.out_num_caps, self.in_num_caps).cuda()

        for i in range(self.routings):
            c = F.softmax(b, dim=1)

            if i == self.routings - 1:
                outputs = squash(torch.sum(c[:, :, :, None] * x_hat, dim=-2, keepdim=True))
            else:
                outputs = squash(torch.sum(c[:, :, :, None] * x_hat_detached, dim=-2, keepdim=True))
                b = b + torch.sum(outputs * x_hat_detached, dim=-1)

        return torch.squeeze(outputs, dim=-2)

# Resnet SETUP


In [ ]:
from torchvision import models
from torchvision.models import ResNet18_Weights, ResNet101_Weights, ResNet50_Weights, ResNet34_Weights
class ResNetLayers(nn.Module):
	def __init__(self, is_freeze=True):
		super(ResNetLayers, self).__init__()
		self.model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
		delattr(self.model, 'layer4')
		delattr(self.model, 'avgpool')
		delattr(self.model, 'fc')

		if is_freeze:
			for index, p in enumerate(self.model.parameters()):
				if index == 15:
					break
				p.requires_grad = False

	def forward(self, x):
		output = self.model.conv1(x)
		output = self.model.bn1(output)
		output = self.model.relu(output)
		output = self.model.layer1(output)
		output = self.model.layer2(output)
		output = self.model.layer3(output)
		return output


class VGGLayers(nn.Module):
	def __init__(self, is_freeze=True):
		super(VGGLayers, self).__init__()
		self.model = models.vgg11(weights=True).features[:11]

		if is_freeze:
			for i in range(4):
				for p in self.model[i].parameters():
					p.requires_grad = False

	def forward(self, x):
		# x = [B, 3, 224, 224]
		return self.model(x)  # [B, 256, 20, 20]


backbone = {'vgg': VGGLayers, 'resnet': ResNetLayers}


class MECapsuleNet(nn.Module):
	"""
	A Capsule Network on Micro-expression.
	:param input_size: data size = [channels, width, height]
	:param classes: number of classes
	:param routings: number of routing iterations
	Shape:
		- Input: (batch, channels, width, height), optional (batch, classes) .
		- Output:((batch, classes), (batch, channels, width, height))
	"""

	def __init__(self, input_size, classes, routings, conv_name='resnet', is_freeze=True):
		super(MECapsuleNet, self).__init__()
		self.input_size = input_size
		self.classes = classes
		self.routings = routings

		self.conv = backbone[conv_name](is_freeze)
		# self.channel_reducer = nn.Conv2d(1024, 256, kernel_size=1)  # 1x1 convolution
		self.conv1 = nn.Conv2d(1024, 256, kernel_size=9, stride=1, padding=0)

		self.primarycaps = PrimaryCapsule(256, 32 * 8, 8, kernel_size=9, stride=2, padding=0)

		self.digitcaps = MECapsule(in_num_caps=32 * 6 * 6,
		                           in_dim_caps=8,
		                           out_num_caps=self.classes,
		                           out_dim_caps=16,
		                           routings=routings)

		self.relu = nn.ReLU()

	def forward(self, x, y=None):
		x = self.conv(x)
		# x = self.channel_reducer(x)  # Reduce channels before passing to conv1
		x = self.relu(self.conv1(x))
		x = self.primarycaps(x)
		x = self.digitcaps(x)
		length = x.norm(dim=-1)
		return length

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.metrics import recall_score, f1_score

class Meter(object):

	def __init__(self):
		"""
		To record the measure the performance.
		"""
		self.Y_true = np.array([], dtype=np.int64)
		self.Y_pred = np.array([], dtype=np.int64)


	def add(self, y_true, y_pred, verbose=False):
		if len(self.Y_true.shape) != len(y_true.shape):
			print('shape self.Y_true', self.Y_true.shape)
			print('y_true', y_true.shape)

		self.Y_true = np.concatenate((self.Y_true, y_true))
		self.Y_pred = np.concatenate((self.Y_pred, y_pred))

	def reset(self):
		self.Y_true = np.array([], dtype=np.int64)
		self.Y_pred = np.array([], dtype=np.int64)

	def value(self):
		eye = np.eye(3, dtype=np.int64)
		Y_true = eye[self.Y_true]
		Y_pred = eye[self.Y_pred]
		uar = recall_score(Y_true, Y_pred, average=None, zero_division=1)
		uf1 = f1_score(Y_true, Y_pred, average=None, zero_division=1)
		return uar, uf1


class UF1(nn.Module):

	def __init__(self):
		super(UF1, self).__init__()


	def forward(self, outputs, targets):
		"""Compute Unweighted F1 score on k folds of LOSO
		Args:
			outputs (list): [k folds]
			targets (list): [k folds]
		Returns:
			UF1 = F1c/C
		"""

		k_folds = len(outputs)
		num_classes = outputs[0].size(1)
		UF1 = 0.0
		for c in range(num_classes):
			TPc, FPc, FNc = 0.0, 0.0, 0.0

			for fold in range(k_folds):
				res = self.compute_on_fold(outputs[fold][:, c], targets[fold][:, c])
				TPc += res[0]
				FPc += res[1]
				FNc += res[2]

			F1c = (2*TPc) / (2 * TPc + FPc + FNc)

		UF1 += F1c / num_classes
		return UF1

	@staticmethod
	def compute_on_fold(output, target):
		"""
		Args
			output (torch.tensor): [1, 0, 1, 1, 1]
			target (torch.tensor): [1, 1, 1, 1, 0]
		Returns:
			(TP, FP, FN): True Positive, False Positive, False Negative
			TP = 3, FP = 1, FN = 1
		"""
		output = output >= 0.5
		target = target >= 0.5

		TP = target.__and__(output).sum()
		FP = (1 - target).__and__(output).sum()
		FN = target.__and__(1 - output).sum()
		return TP.float(), FP.float(), FN.float()


class UARecall(nn.Module):

	def __init__(self):
		super(UARecall, self).__init__()

	def forward(self, outputs, targets):
		"""Compute Unweighted Average Recall
		Args:
			outputs:
			targets:
		Returns:
			UAR = 1/C * sum (Acc_c) where Acc_c is per-class accuracy: Acc_c = TPc/n_c
		"""
		num_classes = outputs.size(1)
		UAR = 0.0

		for c in range(num_classes):
			Acc_c = self.compute_acc(outputs[:, c], targets[:, c])
			UAR += Acc_c / float(num_classes)
		return UAR

	@staticmethod
	def compute_acc(output, target):
		output = output >= 0.5
		target = target >= 0.5

		TP = target.__and__(output).sum()
		Nc = len(target)
		if Nc == 0:
			print('Nc = 0! and TP = ', TP)
		else:
			print("NC is not Zero", Nc, 'while TP=', TP)
		acc = TP.float() / Nc
		return acc

In [ ]:
def me_loss(y_true, y_pred):
    """
    The loss function
    :param y_true: (tensor) of shape [N, num_classes]
    :param y_pred: (tensor) of shape [N, num_classes]
    :return:
    """
    L = y_true * torch.clamp(0.99 - y_pred, min=0.) ** 2 + \
        0.5 * (1 - y_true) * torch.clamp(y_pred - 0.01, min=0.) ** 2

    # class_weights = torch.tensor([1.0, 10.97, 8.43]).cuda()
    # L = L * class_weights

    L_margin = L.sum(dim=1).mean()
    # print('loss:', L.sum(dim=0))
    return L_margin

In [ ]:
def one_hot_encode(labels, num_classes=None):
	if num_classes is None:
		num_classes = len(labels.unique())

	return torch.eye(num_classes, dtype=torch.long).index_select(dim=0, index=labels)

In [ ]:
import os
import sys
import random

import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

from PIL import Image


class Dataset(data.Dataset):

	def __init__(self, root, img_paths, img_labels, transform=None, get_aux=False, aux=None):
		"""Load image paths and labels from gt_file"""
		self.root = root
		self.transform = transform
		self.get_aux = get_aux
		self.img_paths = img_paths
		self.img_labels = img_labels
		self.aux = aux

	def __getitem__(self, idx):
		"""Load image.
		Args:
			idx (int): image idx.
		Returns:
			img (tensor): image tensor
		"""
		img_path = self.img_paths[idx]
		img = Image.open(os.path.join(self.root, img_path)).convert('RGB')
		label = self.img_labels[idx]

		if self.transform:
			img = self.transform(img)

		if not self.get_aux:
			return img, label
		else:
			return img, label, self.aux[idx]

	def __len__(self):
		return len(self.img_paths)


def get_triple_meta_data(file_path):
	df = pd.read_csv(file_path)
	on_paths = list(df.on_frame_path)
	apex_paths = list(df.apex_frame_path)
	off_paths = list(df.off_frame_path)

	paths = [(on, apex, off) for (on, apex, off) in zip(on_paths, apex_paths, off_paths)]
	labels = list(df.label)
	return paths, labels


def get_meta_data(df):
	paths = list(df.apex_frame_path)
	labels = list(df.label)

	return paths, labels


def data_split(file_path, subject_out_idx=0):
	"""Split dataset into train set and validation set
	"""
	# data, subject, clipID, label, apex_frame, apex_frame_path
	data_sub_column = 'subject'

	df = pd.read_csv(file_path)
	subject_list = list(df[data_sub_column].unique())

	subject_out = subject_list[subject_out_idx]
	print('subject_out', subject_out)
	df_train = df[df[data_sub_column] != subject_out]
	df_val = df[df[data_sub_column] == subject_out]

	return df_train, df_val


def upsample_subdata(df, df_four, number=4):
    result = df.copy()
    for i in range(df.shape[0]):
        quotient = number // 1
        remainder = number % 1
        remainder = 1 if np.random.rand() < remainder else 0
        value = quotient + remainder

        tmp = df_four[df_four['subject'] == df.iloc[i]['subject']]
        tmp = tmp[tmp['clip'] == df.iloc[i]['clip']]
        value = min(value, tmp.shape[0])
        tmp = tmp.sample(int(value))
        result = pd.concat([result, tmp])
    return result


def sample_data(df, df_four):


  df_neg = df[df.label == 0]
  df_pos = df[df.label == 1]
  df_sur = df[df.label == 2]

  num_sur = 4
  num_pos = 5 * df_sur.shape[0] / df_pos.shape[0] - 1
  num_neg = 5 * df_sur.shape[0] / df_neg.shape[0] - 1

  df_neg = upsample_subdata(df_neg, df_four, num_neg)
  df_pos = upsample_subdata(df_pos, df_four, num_pos)
  df_sur = upsample_subdata(df_sur, df_four, num_sur)
  print('df_neg', df_neg.shape)
  print('df_pos', df_pos.shape)
  print('df_sur', df_sur.shape)

  df = pd.concat([df_neg, df_pos, df_sur])
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


# Transforms

In [ ]:
import torch
import pandas as pd
from sklearn.utils import shuffle
from torchvision import transforms
from torchvision.transforms import AutoAugmentPolicy


# data_apex_frame_path = '/content/gdrive/MyDrive/TCC/TCC DATABASES/casme_apexFINAL.csv'
# data_four_frames_path = '/content/gdrive/MyDrive/TCC/TCC DATABASES/TesteFinal.csv'
# data_apex_frame_path = '/content/gdrive/MyDrive/TCC/TCC DATABASES/data_four_framesC.csv'
data_four_frames_path = '/content/gdrive/MyDrive/TCC/TCC DATABASES/data_four_framesC.csv'
data_apex_frame_path = '/content/gdrive/MyDrive/TCC/TCC DATABASES/data_four_framesC_editado.csv'
data_root = '/content/gdrive/MyDrive/'
num_classes = 3
batch_size = 32
lr = 0.0001
lr_decay_value = 0.9
epochs = 30
x_meter = Meter()
batches_scores = []


def load_me_data(data_root, file_path, subject_out_idx, batch_size=32, num_workers=2):
	df_train, df_val = data_split(file_path, subject_out_idx)
	df_four = pd.read_csv(data_four_frames_path)
	df_train_sampled = sample_data(df_train, df_four)
	df_train_sampled = shuffle(df_train_sampled)

	train_paths, train_labels = get_meta_data(df_train_sampled)


	# #Trivial Augment
	# train_transforms = transforms.Compose([transforms.Resize((234, 240)),
	#                                        transforms.CenterCrop((224, 224)),
	# 																			 transforms.TrivialAugmentWide(),
	# 																			 transforms.ToTensor(),
	# 																			 transforms.Normalize( mean = (0.485, 0.456, 0.406) , std= (0.229, 0.224, 0.225))])




	# train_transforms = transforms.Compose([transforms.Resize((234, 240)),
	#                                        transforms.RandomCrop((224, 224)),
	# 																			 transforms.RandomRotation(degrees=(-8, 8)),
	# 																			 transforms.RandAugment(2,7),
	#                                        transforms.ToTensor(),
	# 																			 transforms.Normalize( mean = (0.485, 0.456, 0.406) , std= (0.229, 0.224, 0.225))])


  # No AU

	# train_transforms = transforms.Compose([transforms.Resize((234, 240)),
	#                                        transforms.CenterCrop((224, 224)),
	#                                        transforms.ToTensor(),
	# 																			 transforms.Normalize( mean = (0.485, 0.456, 0.406) , std= (0.229, 0.224, 0.225))])



  # Light AU - 3 parameters
	# train_transforms = transforms.Compose([transforms.Resize((234, 240)),
	#                                        transforms.RandomRotation(degrees=(-8, 8)),
	#                                        transforms.RandomHorizontalFlip(),
	#                                        transforms.ColorJitter(brightness=0.2, contrast=0.2,
	#                                                               saturation=0.2, hue=0.2),
	#                                        transforms.RandomCrop((224, 224)),
	#                                        transforms.ToTensor(),
	# 																			 transforms.Normalize( mean = (0.485, 0.456, 0.406) , std= (0.229, 0.224, 0.225))])

	#High AU
	train_transforms = transforms.Compose([transforms.Resize((234, 240)),
	                                       transforms.RandomRotation(degrees=(-8, 8)),
	                                       transforms.RandomHorizontalFlip(),
	                                       transforms.ColorJitter(brightness=0.2, contrast=0.2,
	                                                              saturation=0.2, hue=0.2),
	                                       transforms.RandomCrop((224, 224)),
																				 transforms.RandomGrayscale(p=0.2),
																				 transforms.RandomEqualize(p = 0.1),
																				 transforms.RandomAutocontrast(p = 0.1),
	                                       transforms.ToTensor(),
																				 transforms.Normalize( mean = (0.485, 0.456, 0.406) , std= (0.229, 0.224, 0.225))])



	train_dataset = Dataset(root=data_root,
	                        img_paths=train_paths,
	                        img_labels=train_labels,
	                        transform=train_transforms)

	val_transforms = transforms.Compose([transforms.Resize((234, 240)),
	                                     transforms.CenterCrop((224, 224)),
	                                     transforms.ToTensor(),
																			 transforms.Normalize( mean = (0.485, 0.456, 0.406) , std= (0.229, 0.224, 0.225))])


	val_paths, val_labels = get_meta_data(df_val)

	val_dataset = Dataset(root=data_root,
	                      img_paths=val_paths,
	                      img_labels=val_labels,
	                      transform=val_transforms)

	train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
	                                           batch_size=batch_size,
	                                           num_workers=num_workers,
	                                           shuffle=True)

	val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
	                                         batch_size=batch_size,
	                                         num_workers=num_workers,
	                                         shuffle=False)
	return train_loader, val_loader

# Train test SETUP

In [ ]:
def on_epoch(model, optimizer, lr_decay, train_loader, test_loader, epoch):
	model.train()

	train_loss = 0.0
	correct = 0.
	meter = Meter()

	steps = len(train_loader.dataset) // batch_size + 1
	with tqdm(total=steps) as progress_bar:
		for i, (x, y) in enumerate(train_loader):  # batch training
			y = torch.zeros(y.size(0), num_classes).scatter_(1, y.view(-1, 1),
			                                                 1.)  # change to one-hot coding
			x, y = x.cuda(), y.cuda()  # convert input data to GPU Variable

			optimizer.zero_grad()  # set gradients of optimizer to zero
			y_pred = model(x, y)  # forward
			loss = me_loss(y, y_pred)  # compute loss
			loss.backward()  # backward, compute all gradients of loss w.r.t all Variables
			train_loss += loss.item() * x.size(0)  # record the batch loss
			optimizer.step()  # update the trainable parameters with computed gradients


			y_pred = y_pred.data.max(1)[1]
			y_true = y.data.max(1)[1]

			meter.add(y_true.cpu().numpy(), y_pred.cpu().numpy())
			correct += y_pred.eq(y_true).cpu().sum()

			progress_bar.set_postfix(loss=loss.item(), correct=correct)
			progress_bar.update(1)

		lr_decay.step()  # decrease the learning rate by multiplying a factor `gamma`
		train_loss /= float(len(train_loader.dataset))
		train_acc = float(correct.item()) / float(len(train_loader.dataset))
		scores = meter.value()
		meter.reset()
		print('Training UAR: %.4f' % (scores[0].mean()), scores[0])
		print('Training UF1: %.4f' % (scores[1].mean()), scores[1])

	correct = 0.
	test_loss = 0.

	model.eval()
	for i, (x, y) in enumerate(test_loader):  # batch training
		y = torch.zeros(y.size(0), num_classes).scatter_(1, y.view(-1, 1),
		                                                 1.)  # change to one-hot coding
		x, y = x.cuda(), y.cuda()  # convert input data to GPU Variable

		y_pred = model(x, y)  # forward
		loss = me_loss(y, y_pred)  # compute loss
		test_loss += loss.item() * x.size(0)  # record the batch loss

		y_pred = y_pred.data.max(1)[1]
		y_true = y.data.max(1)[1]

		meter.add(y_true.cpu().numpy(), y_pred.cpu().numpy())
		correct += y_pred.eq(y_true).cpu().sum()

		if (epoch + 1) % 10 == 0 and i % steps == 0:
			print('y_true\n', y_true[:30])
			print('y_pred\n', y_pred[:30])

	print('y_true', y.sum(dim=0))
	scores = meter.value()
	print('Testing UAR: %.4f' % (scores[0].mean()), scores[0])
	print('Testing UF1: %.4f' % (scores[1].mean()), scores[1])

	test_loss /= float(len(test_loader.dataset))
	test_acc = float(correct.item()) / float(len(test_loader.dataset))
	return train_loss, train_acc, test_loss, test_acc, meter


In [ ]:
!pip3 install adamp

  Preparing metadata (setup.py) ... done
  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5984 sha256=4ef1b3b37eabf2db68ab6e8f7e88041c1087be9b7b11691c404a9919db84261d
  Stored in directory: /root/.cache/pip/wheels/33/f9/d6/b2ed816e1f321f6dcf72a99c954223b1259477095f40434979
Successfully built adamp


In [ ]:
from adamp import AdamP


In [ ]:
def train_eval(subject_out_idx):
	best_val_uf1 = 0.0
	best_val_uar = 0.0

	# Model & others
	model = MECapsuleNet(input_size=(3, 224, 224), classes=num_classes, routings=3, is_freeze=True)
	model.cuda()
	optimizer = AdamP(model.parameters(), lr=lr)
	lr_decay = lr_scheduler.ExponentialLR(optimizer, gamma=lr_decay_value)

	for epoch in range(epochs):
		train_loader, test_loader = load_me_data(data_root, data_apex_frame_path,
		                                         subject_out_idx=subject_out_idx,
		                                         batch_size=batch_size)
		train_loss, train_acc, test_loss, test_acc, meter = on_epoch(model, optimizer, lr_decay,
		                                                              train_loader, test_loader,
		                                                              epoch)

		print("==> Subject out: %02d - Epoch %02d: loss=%.5f, train_acc=%.5f, val_loss=%.5f, "
		      "val_acc=%.4f"
		      % (subject_out_idx+1, epoch, train_loss, train_acc,
		         test_loss, test_acc))

		scores = meter.value()
		if scores[1].mean() >= best_val_uf1:
			best_val_uar = scores[0].mean()
			best_val_uf1 = scores[1].mean()
			Y_true = meter.Y_true.copy()
			Y_pred = meter.Y_pred.copy()

	x_meter.add(Y_true, Y_pred, verbose=True)

	return best_val_uar, best_val_uf1

# Training and Results

In [ ]:
import random
import time
inicio = time.time()
import pickle
import matplotlib.pyplot as plt

name_instance = 'scores_ADAMP_AURandom27Normalize_Resnet50_1.pkl'

if __name__ == '__main__':
    uar_list = []  # Lista para armazenar os UARs de cada sujeito
    uf1_list = []  # Lista para armazenar os UF1s de cada sujeito
    subject_indices = list(range(25))  # Lista para os indices dos subjects
    random.shuffle(subject_indices)  # randomizando a lista para treinamento aleatorio
    for i in subject_indices:
        scores = train_eval(subject_out_idx=i)
        batches_scores.append(scores)
        x_scores = x_meter.value()
        current_uar = x_scores[0].mean()  # Calcula a média do UAR
        current_uf1 = x_scores[1].mean()  # Calcula a média do UF1
        uar_list.append(current_uar)  # Armazena o valor
        uf1_list.append(current_uf1)  # Armazena o valor

        print('final uar', x_scores[0], current_uar)
        print('final uf1', x_scores[1], current_uf1)
        print('---- NEXT ---- \n\n')
    with open(name_instance, 'wb') as file:
      data = dict(meter=x_meter, batches_scores=batches_scores, Y_pred = x_meter.Y_pred, Y_true = x_meter.Y_true)
      pickle.dump(data, file)

fim = time.time()
duracao = fim - inicio
print(f"Tempo de execução: {duracao:.4f} segundos")

In [ ]:
 # Criação do gráfico UAR x Subject
subjects = range(1, 26)  # Sujeitos de 1 a 25
plt.figure(figsize=(10, 5))
plt.plot(subjects, uar_list, marker='o', linestyle='-', linewidth=2, markersize=8)
plt.xlabel('Número da Iteração LOSO', fontsize=12)
plt.ylabel('UAR', fontsize=12)
plt.title('UAR por Iteração LOSO ', fontsize=14)
plt.xticks(subjects)  # Garante todos os sujeitos no eixo
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('UAR_por_IT.png')  # Salva a figura
plt.show()

In [ ]:
 # Criação do gráfico UF1 x Subject
subjects = range(1, 26)  # Sujeitos de 1 a 25
plt.figure(figsize=(10, 5))
plt.plot(subjects, uf1_list, marker='o', linestyle='-', linewidth=2, markersize=8)
plt.xlabel('Número da Iteração LOSO', fontsize=12)
plt.ylabel('UF1', fontsize=12)
plt.title('UF1 por Iteração LOSO ', fontsize=14)
plt.xticks(subjects)  # Garante todos os sujeitos no eixo
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('UF1_por_IT.png')  # Salva a figura
plt.show()

In [ ]:
from google.colab import files
files.download(name_instance)

In [ ]:
def print_confusionMatrix(y_true, y_pred):
  from sklearn.metrics import confusion_matrix
  import seaborn as sns
  import matplotlib.pyplot as plt
  conf_matrixAdmp = confusion_matrix(y_true, y_pred)
  ax = sns.heatmap(conf_matrixAdmp, annot=True, cmap='Blues')

  ax.set_title('Seaborn Confusion Matrix with labels\n\n');
  ax.set_xlabel('\nPredicted Values')
  ax.set_ylabel('Actual Values ');

  ## Ticket labels - List must be in alphabetical order
  ax.xaxis.set_ticklabels(['Positivo','Negativo','Surpresa'])
  ax.yaxis.set_ticklabels(['Positivo','Negativo','Surpresa'])

  ## Display the visualization of the Confusion Matrix.
  plt.show()


In [ ]:
from sklearn.metrics import accuracy_score
path = '/content/'+ name_instance
with open(path, "rb") as arquivo:
   e = pickle.load(arquivo)
Y_trueAdmp, Y_predAdmp = e['Y_true'], e['Y_pred']
acc = accuracy_score(Y_trueAdmp, Y_predAdmp)
uar, uf1 = e['meter'].value()
print(uar,uar.mean())
print(uf1,uf1.mean())
print(acc)
print_confusionMatrix(Y_trueAdmp, Y_predAdmp)
